# Vector Calculus Operators

In [1]:
%load_ext autoreload
%autoreload 2

import holoviews as hv
import numpy as np

import uxarray as ux

In [2]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)

In [3]:
uxds["latField"] = uxds.uxgrid.face_lat
uxds["lonField"] = uxds.uxgrid.face_lon

In [4]:
def plot_gradient(uxda_grad, **kwargs):
    uxgrid = uxda_grad.uxgrid
    mag = np.hypot(uxda_grad.zonal_gradient, uxda_grad.meridional_gradient)
    angle = np.arctan2(uxda_grad.meridional_gradient, uxda_grad.zonal_gradient)

    return hv.VectorField((uxgrid.face_lon, uxgrid.face_lat, angle, mag), **kwargs)

## Gradient

In [25]:
grad = uxds["latField"].gradient()
grad.uxgrid.plot() * plot_gradient(grad).opts(
    height=500, width=1000, title="outCSne30 Latitude Field"
)

:Overlay
   .Path.I        :Path   [Longitude,Latitude]
   .VectorField.I :VectorField   [x,y]   (Angle,Magnitude)

In [7]:
sub = uxds["latField"].subset.bounding_circle((-135, 35), r=10)

In [6]:
# del sub.uxgrid._ds['face_edge_connectivity']
# del sub.uxgrid._ds['node_edge_connectivity']
# del sub.uxgrid._ds['edge_node_connectivity']

KeyError: 'face_edge_connectivity'

In [ ]:
# To be considered on a boundary
# - for each edge for the current face, edge_face_connectivity should not contain any fill values
# -

In [8]:
grad = sub.gradient()
grad.uxgrid.plot() * plot_gradient(grad).opts(
    height=500, width=1000, title="outCSne30 Latitude Field"
)

:Overlay
   .Path.I        :Path   [Longitude,Latitude]
   .VectorField.I :VectorField   [x,y]   (Angle,Magnitude)

In [ ]:
sub.uxgrid.node_face_connectivity.values

In [ ]:
sub.uxgrid.face_node_connectivity.values

In [ ]:
grad["zonal_gradient"].plot() * grad.uxgrid.plot(title="Zonal Gradient")

In [ ]:
grad["meridional_gradient"].plot() * grad.uxgrid.plot(title="Meridional Gradient")